In [ ]:
import ...

In [ ]:
now = datetime.now().strftime("%d.%m.%Y")
yest = (datetime.now() - timedelta(days=1)).strftime("%d.%m.%Y")

In [ ]:
folder = r''
filename = '' + yest + '.xlsx'
file_path = folder + filename
data1 = pd.read_excel(file_path)

filename2 = '' + now + '.XLSX'
file_path2 = folder + filename2
data2 = pd.read_excel(file_path2)

filename3 = '' + yest + '.XLSX'
file_path3 = folder + filename3
data3 = pd.read_excel(file_path3)

In [ ]:
data4 = pd.read_excel(r'', sheet_name = '')

In [ ]:
data4

In [ ]:
data1

In [ ]:
data2

In [ ]:
data3

In [ ]:
data1.columns = data1.iloc[0]
data1 = data1.iloc [1: , :]

In [ ]:
data1

In [ ]:
data1['К перечислению за товар, руб.'].sum()

In [ ]:
data5 = pd.read_excel(r'', sheet_name = '')

In [ ]:
data5

In [ ]:
data5 = data5.iloc [1: , :]
data5 = data5.reset_index()
data5 = data5.iloc [: , 1:]
data5.columns = data5.iloc[0]
data5 = data5.iloc [1: , :]
data5 = data5.reset_index()
data5.drop (columns=data5.columns [0], axis= 1 , inplace= True )

In [ ]:
data5

In [ ]:
data5 = data5[['Артикул','Цветомодель','Wildberries\nНоменклатура']]

In [ ]:
data5 = data5.rename(columns={'Wildberries\nНоменклатура': 'Артикул WB'})

In [ ]:
data5

In [ ]:
#data5 = data5.drop_duplicates('Номенклатура')

In [ ]:
#data5

In [ ]:
data2

In [ ]:
a = data2[['Артикул WB','Размер вещи']]
a = a.merge(data5, on = ['Артикул WB'], how = 'left')

In [ ]:
a

In [ ]:
data3

In [ ]:
data3 = data3[['Артикул WB','Баркод','Размер вещи','В пути до клиента', 'Итого по складам']]
data3 = data3.rename(columns={'В пути до клиента': 'Товары в пути_вчера','Итого по складам': 'Итого по складам_вчера'})

In [ ]:
a = a.merge(data3, on = ['Артикул WB','Размер вещи'], how = 'left')
#a = a.drop_duplicates(['Номенклатура','Размер вещи'])
a = a.fillna(0)
a['Итого_сток_вчера'] = a['Товары в пути_вчера']+a['Итого по складам_вчера']

In [ ]:
a

In [ ]:
a.query('Артикул == 0')

In [ ]:
data2

In [ ]:
data2 = data2[['Артикул WB','Размер вещи','В пути до клиента', 'Итого по складам']]
data2 = data2.rename(columns={'В пути до клиента': 'Товары в пути'})
a = a.merge(data2, on = ['Артикул WB','Размер вещи'], how = 'left')
#a = a.drop_duplicates(['Номенклатура','Размер вещи'])
a = a.fillna(0)
a['Итого_сток'] = a['Товары в пути']+a['Итого по складам']

In [ ]:
a

In [ ]:
a = a.fillna(0)
a['Товары в пути_дельта'] = a['Товары в пути']-a['Товары в пути_вчера']
a['Итого по складам_дельта'] = a['Итого по складам']-a['Итого по складам_вчера']
a['Итого_сток_дельта'] = a['Итого_сток']-a['Итого_сток_вчера']

In [ ]:
a

In [ ]:
data1 = data1[['Артикул WB', 'Размер','шт.','Выкупили, шт.','К перечислению за товар, руб.']]

In [ ]:
data1.columns = ['Артикул WB','Размер вещи','Поступления на склад WB','Заказано шт.','Выкупили, шт.', 'Gross Sales']

In [ ]:
data1

In [ ]:
data1 = data1.groupby(by=['Артикул WB','Размер вещи']).sum()

In [ ]:
data1

In [ ]:
data1=data1.reset_index()

In [ ]:
data1['Выкупили, шт.'].sum()

In [ ]:
#data1 = data1.drop_duplicates()

In [ ]:
a = a.merge(data1, on = ['Артикул WB','Размер вещи'], how = 'left')

In [ ]:
a


In [ ]:
#a = a.drop_duplicates(['Номенклатура','Размер вещи'])

In [ ]:
a = a.fillna(0)

In [ ]:
data4.columns = ['Артикул','VAT','COST']

In [ ]:
a = a.merge(data4,on='Артикул', how = 'left')

In [ ]:
a

In [ ]:
a['Net Sales'] = a['Gross Sales']*(1-(a['VAT']-1))

In [ ]:
a

In [ ]:
a['COGS'] = a['Выкупили, шт.']*a['COST']

In [ ]:
a

In [ ]:
a['GP'] = a['Net Sales']-a['COGS']

In [ ]:
a['GPM'] = a['GP']/a['Net Sales']

In [ ]:
a

In [ ]:
a = a.fillna(0)

In [ ]:
a

In [ ]:
a['Расчетный сток'] =a['Итого_сток_вчера']+a['Поступления на склад WB']-a['Заказано шт.']

In [ ]:
a

In [ ]:
a['Проверка текущего стока'] = a['Итого_сток'] - a['Расчетный сток']

In [ ]:
a

In [ ]:
a.info()

In [ ]:
def Статус_по_стоку(a):
    Статус_по_стоку = []
    for i in range(len(a)):
        
        if   a.iloc[i,15] == 0 :
            Статус_по_стоку.append("Продаж нет")
             
        elif a.iloc[i,15] > 0 and  a.iloc[i,25] > 0:
            Статус_по_стоку.append("Фактический сток больше")
            
        elif a.iloc[i,15] > 0 and  a.iloc[i,25] < 0:
            Статус_по_стоку.append("Фактический сток меньше")
            
        elif a.iloc[i,25]  == 0:
            Статус_по_стоку.append("Фактический сток = Расчетный сток")
            
        
        else:
            Статус_по_стоку.append('Не определено')
             
    a['Статус_по_стоку'] =  Статус_по_стоку

Статус_по_стоку(a)


In [ ]:
a

In [ ]:
a.groupby(by=['Статус_по_стоку']).count()

In [ ]:
a = a.drop_duplicates(['Артикул WB','Размер вещи'])

In [ ]:
pd.set_option("display.float_format", "{:.2f}".format)

In [ ]:
a

In [ ]:
a['GPM'].mean()*100

In [ ]:
a['Net Sales'].sum()

In [ ]:

a['Gross Sales'].sum()

In [ ]:
a

In [ ]:
a = a.drop(a.columns[18:20], axis=1)

In [ ]:
a.info()